In [1]:
from tensorflow.keras import models, layers, activations, \
    optimizers, utils, losses, initializers, metrics, callbacks

In [2]:
epochs = 100
batch_size = 32
patience = 3
learning_rate = 0.01
model_path = './model/sigmoid.keras'

In [3]:
model = models.Sequential([
    layers.Resizing(128,128),
    layers.Rescaling(1.0/255),
    layers.BatchNormalization(),

    layers.Conv2D(32,(5,5),
        activation='relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(64,(3,3),
        activation='relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2,2)),
    layers.BatchNormalization(),

    layers.Conv2D(128,(3,3),
        activation='relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2,2)),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128,
        activation = 'tanh',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.Dense(64,
        activation = 'tanh',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.Dense(32,
        activation = 'tanh',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.Dense(5,
        activation = 'softmax',
        kernel_initializer = initializers.RandomNormal()
    )
])

In [4]:
model.compile(
    optimizer = optimizers.Adam(
        learning_rate = learning_rate
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy(name="accuracy")]
)

In [5]:
train = utils.image_dataset_from_directory(
    'Data',
    validation_split = 0.2,
    subset = 'training',
    seed = 123,
    shuffle = True,
    image_size = (512, 512),
    batch_size = 32
)
test =  utils.image_dataset_from_directory(
    'Data',
    validation_split = 0.2,
    subset = 'validation',
    seed = 123,
    shuffle = True,
    image_size = (512, 512),
    batch_size = 32
)
train.class_names


Found 3276 files belonging to 5 classes.
Using 2621 files for training.
Found 3276 files belonging to 5 classes.
Using 655 files for validation.


['hammer', 'nothing', 'pliers', 'screwdriver', 'wrench']

In [6]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 47s 447ms/step - accuracy: 0.2901 - loss: 1.5649 - val_accuracy: 0.1542 - val_loss: 1.6025
Epoch 2/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 37s 451ms/step - accuracy: 0.3821 - loss: 1.4517 - val_accuracy: 0.1618 - val_loss: 1.6305
Epoch 3/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 40s 439ms/step - accuracy: 0.4303 - loss: 1.3516 - val_accuracy: 0.3405 - val_loss: 1.3600
Epoch 4/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 40s 423ms/step - accuracy: 0.4949 - loss: 1.2694 - val_accuracy: 0.4214 - val_loss: 1.2575
Epoch 5/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 36s 437ms/step - accuracy: 0.5023 - loss: 1.2182 - val_accuracy: 0.4641 - val_loss: 1.2082
Epoch 6/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 40s 426ms/step - accuracy: 0.5079 - loss: 1.1911 - val_accuracy: 0.4870 - val_loss: 1.1668
Epoch 7/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 36s 441ms/step - accuracy: 0.5163 - loss: 1.1510 - val_accuracy: 0.5053 - val_loss: 1.1385
Epoch 8/100
82/82 ━━━━━━━━━━━━━━━━━━━━ 36s 440ms/step - accuracy: 0.5315 - loss: 1.1248 - 

In [1]:
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [5]:
model = load_model(model_path)

In [4]:
def load_img(img_path):
    img = Image.open(img_path)
    img_array = np.asarray(img)
    return np.expand_dims(img_array, axis=0)

In [12]:
# e = []
# e.append(np.argmax(model.predict(load_img("data/hammer/000003.jpg"))))
# e.append(np.argmax(model.predict(load_img("data/nothing/nothing1.jpg"))))
# e.append(np.argmax(model.predict(load_img("data/Pliers/000000.jpg"))))
# e.append(np.argmax(model.predict(load_img("data/screwdriver/000000.jpg"))))
# e.append(np.argmax(model.predict(load_img("data/wrench/000000.jpg"))))

# print(e)
response = ['hammer', 'nothing', 'pliers', 'screwdriver', 'wrench']

res = np.argmax(model.predict(load_img("foto1.jpg")))

print(res)
response[res]



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
0


'hammer'

In [ ]:
import cv2;

cam_port = 0
cap = cv2.VideoCapture(cam_port)

status, photo = cap.read()

cap.release()

# Display the captured photo in a window titled “My Photo”
cv2.imshow("My Photo", photo)

photo_resized = cv2.resize(photo, (224, 224))
photo_array = np.expand_dims(photo_resized, axis=0)

res = np.argmax(model.predict(photo_array))

response[res]



# Wait for 5 seconds before closing the window
cv2.waitKey(5000)

# Close all OpenCV windows
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1
